In [75]:
# Import necessary libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [76]:
# Load the dataset
data = pd.read_csv('dataset.csv')  # Replace 'dataset.csv' with the actual filename



In [77]:
# Prepare the data
X = data.iloc[:, :-1].values  # Input features
y = data.iloc[:, -1].values   # Target variable (degree)

In [78]:
# Scale the input features using MinMaxScaler
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

In [79]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


In [80]:
# Determine the number of unique categories for each feature
num_categories = []
for i in range(X.shape[1]):
    unique_categories = np.unique(X[:, i])
    num_categories.append(len(unique_categories))

In [81]:
# Define the embedding dimension
embedding_dim = 50  # Adjust as needed

In [82]:
# Determine the number of input features
num_features = X.shape[1]


In [83]:
# Determine the number of unique degree categories
num_degrees = len(np.unique(y))


In [84]:
model = Sequential()
model.add(Embedding(input_dim=np.max(num_categories) + 1, output_dim=embedding_dim, input_length=num_features))
model.add(LSTM(units=128, return_sequences=True))  # Add return_sequences=True to pass output to the next LSTM layer
model.add(LSTM(units=64))  # Add an additional LSTM layer
model.add(Dense(units=num_degrees, activation='softmax'))

In [85]:
# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [86]:
# Train the model
model.fit(X_train, y_train, batch_size=64, epochs=1000, validation_data=(X_test, y_test))



Epoch 1/1000
13/13 [==============================] - 13s 354ms/step - loss: 2.5656 - accuracy: 0.0887 - val_loss: 2.5592 - val_accuracy: 0.1000
Epoch 2/1000
13/13 [==============================] - 2s 121ms/step - loss: 2.5600 - accuracy: 0.1000 - val_loss: 2.5597 - val_accuracy: 0.1000
Epoch 3/1000
13/13 [==============================] - 2s 118ms/step - loss: 2.5597 - accuracy: 0.1000 - val_loss: 2.5588 - val_accuracy: 0.1000
Epoch 4/1000
13/13 [==============================] - 2s 121ms/step - loss: 2.5598 - accuracy: 0.1000 - val_loss: 2.5575 - val_accuracy: 0.1000
Epoch 5/1000
13/13 [==============================] - 3s 201ms/step - loss: 2.5581 - accuracy: 0.1013 - val_loss: 2.5589 - val_accuracy: 0.0950
Epoch 6/1000
13/13 [==============================] - 2s 176ms/step - loss: 2.5568 - accuracy: 0.1025 - val_loss: 2.5586 - val_accuracy: 0.1050
Epoch 7/1000
13/13 [==============================] - 2s 129ms/step - loss: 2.5538 - accuracy: 0.0962 - val_loss: 2.5639 - val_accuracy

In [87]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss:.4f}')
print(f'Test Accuracy: {accuracy:.4f}')

7/7 [==============================] - 0s 18ms/step - loss: 7.3839 - accuracy: 0.0850
Test Loss: 7.3839
Test Accuracy: 0.0850


In [ ]:
# Mapping of degree class indices to their names
degree_mapping = {
    0: "Bachelor of Technology (BTech) Honors in Agriculture and Plantation Engineering",
    1: "Bachelor of Industrial Studies Honors - Agriculture",
    2: "Bachelor of Software Engineering Honours",
    3: "Bachelor of Technology - Electronic and Communication Engineering",
    4: "Bachelor of Technology - Mechanical Engineering",
    5: "Bachelor of Technology - Mechatronics Engineering",
    6: "Bachelor of Technology - Electrical Engineering",
    7: "Bachelor of Technology - Computer Engineering",
    8: "Bachelor of Technology - Civil Engineering",
    9: "Bachelor of Technology Honors in Engineering – Textile & Clothing",
    10: "Bachelor of Industrial Studies Honors – Textile Manufacture Specialization",
    11: "Bachelor of Industrial Studies Honors – Fashion Design and Product Development",
    12: "Bachelor of Industrial Studies Honors – Apparel Production and Management"
}

# Use the trained model for predictions
new_data = np.array([[0, 2, 1, 1, 1, 1, 4, 4, 2, 1, 1, 2, 0, 1, 1, 0, 0, 0]])  # Replace with your own data
new_data_scaled = scaler.transform(new_data)

# Make predictions
predicted_probs = model.predict(new_data_scaled)
top_n_indices = np.argsort(predicted_probs, axis=1)[:, -3:]  # Get the indices of the top 3 predicted classes

# Get the recommended degrees based on the predicted classes
recommended_degrees = [degree_mapping[idx] for idx in top_n_indices[0]]

print("Recommended Degrees:")
for degree in recommended_degrees:
    print(degree)

In [102]:
# Mapping of degree class indices to their names
degree_mapping = {
    0: "Bachelor of Technology (BTech) Honors in Agriculture and Plantation Engineering",
    1: "Bachelor of Industrial Studies Honors - Agriculture",
    2: "Bachelor of Software Engineering Honours",
    3: "Bachelor of Technology - Electronic and Communication Engineering",
    4: "Bachelor of Technology - Mechanical Engineering",
    5: "Bachelor of Technology - Mechatronics Engineering",
    6: "Bachelor of Technology - Electrical Engineering",
    7: "Bachelor of Technology - Computer Engineering",
    8: "Bachelor of Technology - Civil Engineering",
    9: "Bachelor of Technology Honors in Engineering – Textile & Clothing",
    10: "Bachelor of Industrial Studies Honors – Textile Manufacture Specialization",
    11: "Bachelor of Industrial Studies Honors – Fashion Design and Product Development",
    12: "Bachelor of Industrial Studies Honors – Apparel Production and Management"
}

# Use the trained model for predictions
new_data = np.array([[1, 1, 1, 1, 1, 1, 4, 4, 2, 1, 1, 2, 1, 1, 1, 0, 1, 0]])  # Replace with your own data
new_data_scaled = scaler.transform(new_data)

# Make predictions
predicted_probs = model.predict(new_data_scaled)
degree_probs = [(degree_mapping[idx], '{:.10f}'.format(prob)) for idx, prob in enumerate(predicted_probs[0])]

# Sort the degrees based on the probabilities in descending order
sorted_degrees = sorted(degree_probs, key=lambda x: float(x[1]), reverse=True)

print("Recommended Degrees:")
for degree, prob in sorted_degrees:
    print(f"Degree: {degree}, Probability: {prob}")

1/1 [==============================] - 0s 24ms/step
Recommended Degrees:
Degree: Bachelor of Technology - Computer Engineering, Probability: 0.7648050785
Degree: Bachelor of Industrial Studies Honors – Fashion Design and Product Development, Probability: 0.2070324421
Degree: Bachelor of Technology - Mechanical Engineering, Probability: 0.0114820031
Degree: Bachelor of Industrial Studies Honors - Agriculture, Probability: 0.0086052353
Degree: Bachelor of Technology - Electronic and Communication Engineering, Probability: 0.0037605229
Degree: Bachelor of Software Engineering Honours, Probability: 0.0030895849
Degree: Bachelor of Technology (BTech) Honors in Agriculture and Plantation Engineering, Probability: 0.0007370203
Degree: Bachelor of Technology - Electrical Engineering, Probability: 0.0003459659
Degree: Bachelor of Technology Honors in Engineering – Textile & Clothing, Probability: 0.0001176547
Degree: Bachelor of Technology - Civil Engineering, Probability: 0.0000181262
Degree: 

In [105]:
import pickle

# Save the model and scaler in a single file
with open('trained_model_and_scaler.pkl', 'wb') as file:
    pickle.dump((model, scaler), file)

# Load the saved model and scaler
with open('trained_model_and_scaler.pkl', 'rb') as file:
    loaded_model, loaded_scaler = pickle.load(file)